# Cartpole Example
This notebook goes over the process of setting up and solving a trajectory optimization problem using multiple solvers.

## Loading the Required Packages
To set up the problem, we import `TrajectoryOptimization`, along with `StaticArrays` and `LinearAlgebra` to help in the set up.

To keep things simple, we use the cartpole model from the [`RobotZoo`](https://github.com/bjack205/RobotZoo.jl) package.

In [26]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
using TrajectoryOptimization
using RobotDynamics
import RobotZoo.Cartpole
using StaticArrays, LinearAlgebra

  Activating environment at `~/SSD/Code/Julia/tinympc-julia-old/altro_rocket/Project.toml`


In [27]:
import RobotZoo.Pendulum

### Load the model
Here we instantiate our model, which is a sub-type of `AbstractModel`, and extract out the state dimension `n` and control dimension `m`.

In [28]:
model = Pendulum(mass=1., len=1, b=0, lc=1, I=0.25, g=10.)
n,m = size(model);

### Define the model discretization
We now define how we will discretize our model time. Here we choose to solve a problem with 5 second horizon and use 101 knotpoints, for a uniform time step of 0.05 seconds.

In [29]:
N = 101
tf = 5.
dt = tf/(N-1)

0.05

### Set initial and final state
All trajectory optimization problems require an initial condition (it will be assumed to be the origin if not passed to the problem constructor). The final state is not be required, and could be replaced with constraints on the terminal state, in practice. Here we want the cartpole to start with the pendulum hanging down and then swing up. Note that we use `SVector`s here, but the constructor will automatically convert normal vectors, as well.

In [30]:
x0 = @SVector zeros(n)
# xf = @SVector [0, pi, 0, 0];  # i.e. swing up
xf = @SVector [-pi, 0];  # i.e. swing up

### Define the Objective
Here we define a simple LQR tracking objective:
$$ \frac{1}{2} (x_N - x_f)^T Q_f (x_N - x_f) +  \frac{1}{2} \sum_{k=0}^{N-1} (x_k - x_f)^T Q_k (x_k - x_f) + (u_k - u_f)^T R_k (u_k - u_f) $$

Here it is very important to use the `Diagonal` and `SVector` constructors, since this will have a significant impact on performance. To create the objective, we use the convenient `LQRObjective` method.

In [31]:
# Set up
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = LQRObjective(Q,R,Qf,xf,N);

### Define Constraints
We now add constraints to our problem. We place simple bounds on the controls and make the goal state and hard constraint. Note that we first create an empty `ConstraintList`, then create the individual constraints, and finally add the constraints to the list, specifying the knot point indicies to which the constraint applies. Since we don't have controls at the final knot point, we set the indices to `1:N-1`.

In [32]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Control Bounds
u_bnd = 4.0
bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
add_constraint!(conSet, bnd, 1:N-1)

# Goal Constraint
goal = GoalConstraint(xf)
# add_constraint!(conSet, goal, N)

GoalConstraint{2, Float64}(2, [-3.141592653589793, 0.0], [1, 2])

### Define the problem
We now pass all the required information to the `Problem` constructor to define our problem

In [33]:
prob = Problem(model, obj, x0, tf, xf=xf, constraints=conSet);

### Initialization
As is standard for all non-convex optimization problems, initialization is very important. If a good guess isn't available, it is usually best to initialize the system will a set of controls that keep the state bounded or constant. It is often useful to perturb the system slightly, and provide a dynamicall feasible state trajectory. Here we initialize the system will near-zero controls and then simulate the system forward with the `rollout!` method to provide the initial state guess.

In [ ]:
u0 = @SVector fill(0.01,m)
U0 = [u0 for k = 1:N-1]
initial_controls!(prob, U0)
rollout!(prob);

## Solving with ALTRO
Now that the problem is defined, we can use any of the supported solvers to solve the problem. Here we use [`Altro.jl`](https://github.com/RoboticExplorationLab/ALTRO.jl), a very fast specialized solver that uses iterative LQR (iLQR) within an augmented Lagrangrian framework to handle the constraints.

In [ ]:
using Altro
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

altro = ALTROSolver(prob, opts)
set_options!(altro, show_summary=true)
solve!(altro);

We can retrive some basic information about the solve for post-analysis,

In [ ]:
println("max_violation: ", max_violation(altro))
println("cost:          ", cost(altro))
println("iterations:    ", iterations(altro));

and extract the solution from either `prob` or `altro` using `states` and `controls`.

In [ ]:
# Extract the solution
X = states(altro)
U = controls(altro)

These functions return vectors of vectors. If you want to convert them to 2D arrays, simply use `hcat`. If `n`, `m`, or `N` are large, it's usually a good idea to convert it to a standard Julia `Vector` before concatenation.

In [ ]:
using Plots
Un = hcat(Vector.(U)...)
Xn = hcat(Vector.(X)...)
t_vec = range(0,tf,length=N)
display(plot(t_vec,Xn',label = ["x₁" "x₂" "ẋ₁" "ẋ₂"],linestyle=[:solid :solid :solid :dash :dash :dash],
             title = "State History",
             xlabel = "time (s)", ylabel = "x"))
display(plot(t_vec[1:end-1],Un',label = ["u₁" "u₂" "u₃"],
             title = "Input History",
             xlabel = "time (s)", ylabel = "u"))

## Solving with iLQR
If we want to solve the unconstrained problem with iLQR, we simply reset our initial guess and create a new solver:

In [ ]:
ilqr = Altro.iLQRSolver(prob, opts)
initial_controls!(ilqr, U0)
solve!(ilqr);

## Solving with Ipopt
TrajectoryOptimization provides a way to represent a trajectory optimization as a generic nonlinear program (NLP) and solve using solvers that implement the NLP interface in [`MathOptInterface`](https://github.com/jump-dev/MathOptInterface.jl), such as Ipopt (via [`Ipopt.jl`](https://github.com/jump-dev/Ipopt.jl)). 

Before converting to an NLP, we need to explicitly add the dynamics and initial condition as constraints to the optimization problem, which we do via `add_dynamics_constraints!`.

We then convert our `Problem` to a `TrajOptNLP`, converting any applicable constraints to simple bounds on the decision variables (e.g. bound constraints and goal constraints), and specifying that the the constraint Jacobian uses a vector of non-zero values rather than a generic sparse matrix.

In [ ]:
using Ipopt
using MathOptInterface
const MOI = MathOptInterface

# Copy problem to avoid modifying the original problem
prob_nlp = copy(prob)

# Add the dynamics and initial conditions as explicit constraints
TrajectoryOptimization.add_dynamics_constraints!(prob_nlp)

# Reset our initial guess
initial_controls!(prob_nlp, U0)
rollout!(prob_nlp)

# Create the NLP
nlp = TrajOptNLP(prob_nlp, remove_bounds=true, jac_type=:vector);

We can now solve with Ipopt by creating the `MathOptInterface.Optimizer` and setting up the problem in the `MathOptInterface` format using `TrajectoryOptimization.build_MOI!` method

In [ ]:
optimizer = Ipopt.Optimizer()
TrajectoryOptimization.build_MOI!(nlp, optimizer)
MOI.optimize!(optimizer)
MOI.get(optimizer, MOI.TerminationStatus())

Since the solve updates the data structures in `nlp` during the course of the solve, we can use either `optimizer` or `nlp` to get information after the solve.

In [ ]:
println("max_violation: ", max_violation(nlp))
println("cost:          ", cost(nlp));

## Comparing Solve Times
Here we demonstrate the compuational advantage of ALTRO over Ipopt. We demonstrate how to use `BenchmarkTools` to time a solve using `MathOptInterface` by resetting the initial guess between solves. As shown, ALTRO is about 100x faster than Ipopt.

In [ ]:
using BenchmarkTools

# Reset initial guess and then benchmark ALTRO solver
initial_controls!(altro, U0)
b_altro = benchmark_solve!(altro)

In [ ]:
# Reset initial guess and benchmark Ipopt solver
initial_controls!(prob_nlp, U0)
rollout!(prob_nlp)
nlp = TrajOptNLP(prob_nlp, remove_bounds=true, jac_type=:vector)
Z0 = copy(TrajectoryOptimization.get_trajectory(nlp).Z)

optimizer = Ipopt.Optimizer(print_level=0)
TrajectoryOptimization.build_MOI!(nlp, optimizer)
Z = MOI.VariableIndex.(1:length(Z0))
b_ipopt = @benchmark begin
    MOI.optimize!($optimizer)
    MOI.set($optimizer, MOI.VariablePrimalStart(), $Z, $Z0)
    MOI.get($optimizer, MOI.TerminationStatus())
end

In [ ]:
using Statistics
@show cost(nlp)
@show cost(altro)
@show max_violation(nlp)
@show max_violation(altro)
jdg = judge(median(b_altro), median(b_ipopt))
println("Speed improvement: ", round(1/ratio(jdg).time), "x")
jdg

## Visualizing the Results

In [ ]:
using TrajOptPlots
using MeshCat
using Plots

vis = Visualizer()
render(vis)

We can use the default geometries for the robot, as defined in `TrajOptPlots`, using the `set_mesh!` command

In [ ]:
TrajOptPlots.set_mesh!(vis, model)

We can visualize the results of a solver by passing the solver to `visualize!`. In practice, all a solver needs to implement for this method is a `get_model` method that returns an `AbstractModel` and a `get_trajectory` method that returns an `AbstractTrajectory`.

In [ ]:
visualize!(vis, altro);

In [ ]:
visualize!(vis, model, TrajectoryOptimization.get_trajectory(nlp));

We can also compare both solutions by passing in both solvers:

In [ ]:
visualize!(vis, altro, nlp);

Or the model and different trajectories:

In [ ]:
visualize!(vis, model, get_trajectory(altro), get_trajectory(nlp));

After this, we'll have extra geometries floating around, which can be deleting using:

In [ ]:
delete!(vis["robot_copies"]);

### Tip: Defining the visualazation method
Visualization of the model is defined by the `visualize!(vis, model::MyModel, x::AbstractVector)` command:
 
 ```julia
 function TrajOptPlots.visualize!(vis, model::RobotZoo.Cartpole, x::AbstractVector)
    y = x[1]
    θ = x[2]
    q = expm((pi-θ) * @SVector [1,0,0])
    settransform!(vis["robot","cart"], Translation(0,-y,0))
    settransform!(vis["robot","cart","pole","geom"], LinearMap(UnitQuaternion(q)))
end

 ```